In [1]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sbn

import pickle as pkl
import zipfile
from tqdm import tqdm
import math
import copy as cp
import re
import time
import pymorphy2 as pm2

import scipy as sc
import sklearn.cross_validation  as cv
from sklearn.metrics import accuracy_score, log_loss
import sklearn.metrics as metr
import sklearn.ensemble as ens
import sklearn.linear_model as lin
import sklearn.feature_extraction as fe
import sklearn.preprocessing as prep

from functools import reduce
import datetime as dt

import numba as nb
from numba import jit
import sys
sys.stderr.write('all_right')

all_right

# First Data Manipulation

In [ ]:
# Load train data
train = pd.read_csv('data/train.csv')

# Load test data
test = pd.read_csv('data/test.csv')

In [ ]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing as mltp

thPool = ThreadPool(processes=mltp.cpu_count()*100)

# Определяю функции для стемминга и лемматизации (стемминг использовался в самой первой итерации и показал себя хже лемматизации)

import nltk
from numba import jit,autojit

st = nltk.stem.snowball.RussianStemmer()
ma = pm2.MorphAnalyzer()

# Обрабатываю текст объявления, убирая лишние символы и разделяя его на отдельные слова
# Не все спецсимволы учтены
# Также следовало бы учитывать 
#     - смайлики,
#     - размеры (например, 99х99х99)
#     - использование иностранных слов
#     - использование цифр
@jit
def str_proc(x):
    return x.lower().\
replace('. ',' ').replace(', ',' ').replace(': ',' ').replace('; ',' ').replace('! ',' ').replace('? ',' ').\
replace('.',' ').replace(',',' ').replace(':',' ').replace(';',' ').replace('!',' ').replace('?',' ').\
replace('\n',' ').replace('\t',' ').replace('(',' ').replace(')',' ').replace('«',' ').replace('»',' ').replace('"',' ').\
replace('\xa0',' ').replace('#',' ').replace('\\',' ').replace('/',' ').replace('|',' ').replace('*','').\
replace('{',' ').replace('}',' ').replace('[',' ').replace(']',' ').replace('+',' ').replace('&',' ').\
replace('@',' ').replace('№',' ').replace('%',' ').replace('^',' ').replace('<',' ').replace('>',' ').\
split(' ')

def process_string_for_stemming_p(string):
    return list(filter(lambda x: x != '', list(pool.map(lambda x: ma.parse(x)[0].normal_form , str_proc(string)))))

def process_string_for_stemming(string):
    return list(filter(lambda x: x != '', list(map(lambda x: ma.parse(x)[0].normal_form, str_proc(string)))))

train['stem_description'] = list(thPool.map(str_proc, tqdm(train.description),chunksize=3000))
train['stem_title'] = list(thPool.map(str_proc, tqdm(train.title),chunksize=3000))
test['stem_description'] = list(thPool.map(str_proc, tqdm(test.description),chunksize=3000))
test['stem_title'] = list(thPool.map(str_proc, tqdm(test.title),chunksize=3000))

train['stem'] = list(map(lambda x,y: x+y,train.stem_title, train.stem_description))
test['stem'] = list(map(lambda x,y: x+y,test.stem_title, test.stem_description))

In [ ]:
## Make 'word-count' feature

listmerge5=lambda ll: [el for lst in tqdm(ll) for el in lst]

my_stem = list(set(listmerge5(list(train.stem)+list(test.stem))))

my_stem_all = listmerge5(list(train.stem)+list(test.stem))

my_stem_vc = pd.Series(my_stem_all).value_counts()

d = {i:j for i, j in zip(list(set(my_stem)),\
                 list(thPool.map(lambda x:ma.parse(x)[0].normal_form, tqdm(list(set(my_stem))), chunksize= 10000)))}

d_df = pd.DataFrame()

keys_ = [i for i in d.keys()]
values_ = [d[i] for i in d.keys()]

d_df['keys_'] = keys_
d_df['values_'] = values_
d_df['1'] = 1

d_df_filtred = d_df[d_df['keys_'].isin(my_stem_vc[(my_stem_vc >= 5) == True].index) &\
     d_df['values_'].isin(d_df.groupby('values_')['1'].count()[d_df.groupby('values_')['1'].count() > 1].keys())][['values_','keys_']]

d_df_filtred = d_df_filtred.set_index('keys_')

# Выбираю для каждого слова в объявлении лемму, если оно попалов в список d_df_filtred
def import_all_values(lst):
    try:
        return list(d_df_filtred.values_[lst])
    except:
        return []

train['stem_F'] = list(map(lambda x: import_all_values(x), tqdm(train.stem)))
test['stem_F'] = list(map(lambda x: import_all_values(x), tqdm(test.stem)))

# Фильтрую от NaNов и пустых слов
@jit
def filt(x):
    for i in x:
        if isinstance(i,float):
            x.remove(i)
        if i == '':
            x.remove(i)
    return x

train['stem_F'] = list(map(lambda x: filt(x), tqdm(train.stem_F)))
test['stem_F'] = list(map(lambda x: filt(x), tqdm(test.stem_F)))

train['stem_bag'] = list(map(lambda x: ' '.join(list(map(str,x))), tqdm(train.stem_F)))
test['stem_bag'] = list(map(lambda x: ' '.join(list(map(str,x))), tqdm(test.stem_F)))

In [ ]:
train.to_pickle('train_without_stop_26_09.pkl')
test.to_pickle('test_without_stop_26_09.pkl')

# Load data

In [2]:
train = pd.read_pickle('train_without_stop_24_09.pkl')
test = pd.read_pickle('test_without_stop_24_09.pkl')
cat = pd.read_csv('data/category.csv')
def proc_cat(cat):
    l = cat.split('|')
    while len(l) != 4:
        l.append(l[len(l)-1])
    return l
for i in [1,2,3,4]:
    cat[i] = 0
cat.loc[:,[1,2,3,4]] = list(map(lambda x: proc_cat(x), cat.name))

def categorise(series):
    j = 0
    res = series.copy()
    for i in set(series):
        res[series == i] = j
        j = j + 1
    return res

for i in [1,2,3,4]:
    cat[str(i)+'_cat'] = categorise(cat[i])
cat = cat.set_index('category_id')

d_cats = {}
for x in cat.index:
    d_cats.update([(x,{'1_cat':cat.loc[x,'1_cat'],'2_cat':cat.loc[x,'2_cat'],'3_cat':cat.loc[x,'3_cat'],\
                       '4_cat':cat.loc[x,'4_cat']})])

In [3]:
# смотрю, есть ли у меня сохраненная структура с моделями и их скорами
try:
    f = open('my_current_models.pkl','rb')
    models = pkl.load(f)
    f.close()
except:
    cv_ = cv.StratifiedKFold(train.category_id, n_folds=5)
    models = {'cross_val':cv_}

# Transformation data

### Remove stop-words by yandex list

In [16]:
stop_words = ['а','бы','быть','в','весь','вот','все','всей','вы','говорить','да','для','до','еще','же','знать','и','из','к',\
              'как','который','мочь','мы','на','наш','не','него','нее','нет','них','но','о','один','она','они','оно','оный',\
              'от','ото','по','с','свой','себя','сказать','та','такой','только','тот','ты','у','что','это','этот','я','','nan',
              'рубль']

In [19]:
def to_str(l):
    res = list(map(str,l))
    res_2 = []
    for j in res:
        numbers = re_num.findall(j)
        if len(numbers) != 0:
            for numb in numbers:
                j = j.replace(numb,'')
                flag = True
                for i in stop_words:
                    if (i == numb) | (numb == ''):
                        flag = False
                if flag:
                    res_2.append(numb)
            flag = True
            for i in stop_words:
                if (i == j) | (j == ''):
                    flag = False
            if flag:
                res_2.append(j)
        else:
            flag = True
            for i in stop_words:
                if (i == j) | (j == ''):
                    flag = False
            if flag:
                res_2.append(j)
    return res_2

re_num = re.compile('\d{1,10}')
train['stem_F_str'] = list(map(to_str, tqdm(train.stem_F)))
test['stem_F_str'] = list(map(to_str, tqdm(test.stem_F)))

100%|██████████| 243166/243166 [01:03<00:00, 3834.06it/s]


In [20]:
train['stem_F_str_for_BOW'] = list(map(lambda x: ' '.join(x), tqdm(train.stem_F_str)))
test['stem_F_str_for_BOW'] = list(map(lambda x: ' '.join(x), tqdm(test.stem_F_str)))

100%|██████████| 243166/243166 [00:00<00:00, 439266.45it/s]


In [21]:
train = train[['item_id', 'title', 'description', 'price', 'category_id', 'stem_description', 'stem_title', 'stem',
       'stem_F', 'stem_bag', 'stem_F_str', 'stem_F_str_for_BOW']]

In [22]:
train.columns = ['item_id', 'title', 'description', 'price', 'category_id', 'words_sep_descr', 'words_sep_title', 'all_words',
       'all_lemma', 'glue_lemma', 'filtred_lemma', 'filtred_lemma_glue']
test.columns = ['item_id', 'title', 'description', 'price', 'words_sep_descr', 'words_sep_title', 'all_words',
       'all_lemma', 'glue_lemma', 'filtred_lemma', 'filtred_lemma_glue']

In [24]:
train.to_pickle('train_without_stop_26_09.pkl')
test.to_pickle('test_without_stop_26_09.pkl')

## Compute feature
### Bag of words
### TF-IDF

> Выделим из нашего корпуса мешок слов и tf-idf для каждого документа (то есть объявления))

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1)
X_bag = vectorizer.fit_transform(list(train['filtred_lemma_glue'])+ list(test['filtred_lemma_glue']))

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_bag)

In [5]:
X_tfidf_tr = X_tfidf[:train.shape[0],:]
X_tfidf_te = X_tfidf[train.shape[0]:,:]

In [6]:
X_bag_tr = X_bag[:train.shape[0],:]
X_bag_te = X_bag[train.shape[0]:,:]

## Decomposition
### SVD

> Так как деревья плохо работают со спаровыми матрицами, то я разложу нашу матрицу tf-idf для уменьшения размерности

In [7]:
import sklearn.decomposition as dec

In [8]:
svd = dec.TruncatedSVD(n_components=100)

X_svd = svd.fit_transform(X_tfidf)

X_svd_tr = X_svd[:train.shape[0],:]
X_svd_te = X_svd[train.shape[0]:,:]

In [9]:
svd_10 = dec.TruncatedSVD(n_components=10)

X_svd_10 = svd_10.fit_transform(X_tfidf)

X_svd_10_tr = X_svd_10[:train.shape[0],:]
X_svd_10_te = X_svd_10[train.shape[0]:,:]

>По-хорошему, количестов компонентов разложения можно подобрать с помощью кросс-валидации

>Равно как и параметры разложения (или даже его тип)

# Try to classify

### Functions

In [10]:
# обновление структуры с моделью и результатами 
def add_models(models,model,results,name = 'my_current_models.pkl'):
    models.update([(str(dt.datetime.today()),{'model':cp.deepcopy(model),'result':cp.deepcopy(results)})])
    f = open(name,'wb')
    pkl.dump(models,f)
    f.close()
    return models

In [11]:
# рассчет точности по уровням категорий
def accuracy_level(true,predict):
    true_1 = list(map(lambda x: d_cats[x]['1_cat'],true))
    predict_1 = list(map(lambda x: d_cats[x]['1_cat'],predict))
    true_2 = list(map(lambda x: d_cats[x]['2_cat'],true))
    predict_2 = list(map(lambda x: d_cats[x]['2_cat'],predict))
    true_3 = list(map(lambda x: d_cats[x]['3_cat'],true))
    predict_3 = list(map(lambda x: d_cats[x]['3_cat'],predict))
    true_4 = list(map(lambda x: d_cats[x]['4_cat'],true))
    predict_4 = list(map(lambda x: d_cats[x]['4_cat'],predict))
    return [metr.accuracy_score(true_1,predict_1),
           metr.accuracy_score(true_2,predict_2),
           metr.accuracy_score(true_3,predict_3),
           metr.accuracy_score(true_4,predict_4)]

def accuracy_scoring_levels(est,X_true,predict):
    return accuracy_level(est.predict(X_true),predict)    

# подсчет скора по уровням категорий на кросс-валидации, запись в структуру
def result_for_avito(cv_,true,pred,type_of_fea):
    res = []
    for i,j in cv_:
        res.append(accuracy_level(true[j],pred[j]))
    res = np.array(res)
    d = {str(i+1)+'_level': {'score':j[1],'var':j[0]} for i,j in enumerate(zip(res.var(axis = 0),res.mean(axis = 0)))}
    d.update([('type_of_fea',type_of_fea)])
    return d

In [12]:
# Выводит топ моделей
def print_top_models(models, n = 5, lvls = True, lvl = 4):
    for i in models:
        if len(models[i]) == 3:
            del models[i]
            break

    r = []
    for i in models:
        if i != 'cross_val':
            try:
                if lvls == True:
                    r.append([models[i]['model'].get_params(),models[i]['result']['4_level']['score'],\
                              models[i]['result']['3_level']['score'],\
                              models[i]['result']['2_level']['score'],models[i]['result']['1_level']['score']])
                else:
                    r.append([models[i]['model'].get_params(),models[i]['result'][str(lvl)+'_level']['score']])
            except:
                print(i,' ',models[i])

    df = pd.DataFrame(r).sort(1,ascending = False).head(n)
    if lvls == True:
        df.columns = ['Model','Accuracy for 4 level','Accuracy for 3 level','Accuracy for 2 level','Accuracy for 1 level']
    else:
        df.columns = ['Model','Accuracy for '+str(lvl)+' level']
    return df

### Define CV-folds

#### We define cv-folds on point of loading 'my_current_models.pkl'

### Try different models

In [13]:
import sklearn.linear_model as lin
import sklearn.ensemble as ens

In [202]:
model = lin.LogisticRegression(fit_intercept=True,n_jobs=-1)
res = result_for_avito(models['cross_val'],train.category_id, cv.cross_val_predict(model, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                    cv = models['cross_val']),'tf-idf_without_stopwords')
models = add_models(models,model,res)

model = lin.LogisticRegression(fit_intercept=False,n_jobs=-1)
res = result_for_avito(models['cross_val'],train.category_id, cv.cross_val_predict(model, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                    cv = models['cross_val']),'tf-idf_without_stopwords')
models = add_models(models,model,res)
sys.stderr.write('Next!')

Next!

In [203]:
for c_ in tqdm([1.5,2,2.5]):
    LR = lin.LogisticRegression(fit_intercept=False,C = c_,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val']),'tf-idf_without_stopwords')
    models = add_models(models,LR,res)
    
    LR = lin.LogisticRegression(fit_intercept=True,C = c_,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val']),'tf-idf_without_stopwords')
    models = add_models(models,LR,res)
sys.stderr.write('Next!')

100%|██████████| 3/3 [1:02:23<00:00, 1230.12s/it]
Next!

In [ ]:
for c_ in tqdm([3,3.5,4,5,5.5,6,7,8,10]):
    LR = lin.LogisticRegression(fit_intercept=True,C = c_,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id,\
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val']),'tf-idf_without_stopwords')
    models = add_models(models,LR,res)
sys.stderr.write('Next!')

100%|██████████| 9/9 [2:01:13<00:00, 867.55s/it]
Next!

> Можно увидеть, что логистическая регрессия на заданных параметрах с регулируемыми параметрами регуляризации и fit_intercept достигает максимума по сетке при C = 4.0 и fit_intercept = True.

> При этом в топ-8 результатов нет моделей без свободного члена.

> Попробуем посмотреть в окрестности C = 4, а именно на сетке [3.7,4.2,4.5]

In [ ]:
for c_ in tqdm([3.7,4.2,4.5]):
    LR = lin.LogisticRegression(fit_intercept=True,C = c_,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id,\
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val']),'tf-idf_without_stopwords')
    models = add_models(models,LR,res)
sys.stderr.write('Next!')

100%|██████████| 3/3 [36:41<00:00, 730.23s/it]
Next!

> И на чуть более мелкой сетке

In [36]:
for c_ in tqdm([3.8,3.9,4.1]):
    LR = lin.LogisticRegression(fit_intercept=True,C = c_,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id,\
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val']),'tf-idf_without_stopwords')
    models = add_models(models,LR,res)
sys.stderr.write('Next!')

100%|██████████| 3/3 [39:15<00:00, 783.25s/it]
Next!

## Найдем среди моделей LR самые лучшие и посмотрим их скор

In [55]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(lin.LogisticRegression()) )},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True):
    print(models[i]['result']['4_level']['score'],'\t',models[i]['result']['1_level']['score'],'\t',models[i]['model'].get_params()['C'],'\t',\
          models[i]['model'].get_params()['fit_intercept'],'\t',models[i]['result']['type_of_fea'])

0.842087163982 	 0.952763528875 	 3.9 	 True 	 tf-idf_without_stopwords
0.842078991586 	 0.952790091408 	 4 	 True 	 tf-idf_without_stopwords
0.842068777894 	 0.952755353494 	 3.8 	 True 	 tf-idf_without_stopwords
0.842052442804 	 0.952769671809 	 4.1 	 True 	 tf-idf_without_stopwords
0.842044273036 	 0.952775800303 	 4.2 	 True 	 tf-idf_without_stopwords
0.842036094253 	 0.952734928241 	 3.7 	 True 	 tf-idf_without_stopwords
0.841989104524 	 0.952632777766 	 3.5 	 True 	 tf-idf_without_stopwords
0.841982987009 	 0.952859557887 	 4.5 	 True 	 tf-idf_without_stopwords
0.841901266957 	 0.95247344069 	 3 	 True 	 tf-idf_without_stopwords
0.841866538459 	 0.952896327515 	 5 	 True 	 tf-idf_without_stopwords
0.841862451236 	 0.952961700722 	 5.5 	 True 	 tf-idf_without_stopwords
0.841705167165 	 0.95231820246 	 3.9 	 True 	 tf-idf_without_stopwords
0.841639782478 	 0.952906540205 	 6 	 True 	 tf-idf_without_stopwords
0.841580558609 	 0.95223034693 	 2.5 	 True 	 tf-idf_without_stopwords
0.8

> Лучшая логичтическая регрессия имеет коэф регуляризации 3.9 и обучается вместе со свободным членом

> Попробуем увеличить число итераций и понизить tolerance (критерий останова)

In [50]:
LR = lin.LogisticRegression(fit_intercept=True,C = 3.9,n_jobs=-1,tol=0.000001,max_iter = 1000)
res = result_for_avito(models['cross_val'],train.category_id,\
                           cv.cross_val_predict(LR, X_tfidf_tr,train.category_id,n_jobs=-1,\
                                        cv = models['cross_val'],verbose = 2),'tf-idf_without_stopwords')
models = add_models(models,LR,res)
sys.stderr.write('Next!')

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 22.5min finished
Next!

In [51]:
res

{'1_level': {'score': 0.95231820245959287, 'var': 4.8217750614520183e-07},
 '2_level': {'score': 0.92384119332607251, 'var': 1.8064638108986135e-07},
 '3_level': {'score': 0.84622798700725466, 'var': 1.9709426299616991e-07},
 '4_level': {'score': 0.84170516716535171, 'var': 1.3020843780604523e-07},
 'type_of_fea': 'tf-idf_without_stopwords'}

> Как-то неожиданно увеличение числа итераций подбора параметров ухудшило скор (вероятно, это произошло из за низкого толеранса, при котором модель переобучилась)

## Попробуем случайный лес на SVD (100) для tfidf

In [ ]:
for i in tqdm([10,25,50,75,100,150,200,300]):
    RF = ens.RandomForestClassifier(n_estimators=i,max_depth=13,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, X_svd_tr,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_100_tfidf')
    models = add_models(models,RF,res)
sys.stderr.write('Next!')

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
max_depths = [15,18,21,23,24,29,35,40,43,50]
max_depths.reverse()
for i in tqdm(max_depths):
    RF = ens.RandomForestClassifier(n_estimators=75,max_depth=i,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, X_svd_tr,train.category_id,n_jobs=1,\
                                            cv = models['cross_val'],verbose = 2),'svd_100_tfidf')
    models = add_models(models,RF,res)
sys.stderr.write('Next!')

100%|██████████| 10/10 [42:39<00:00, 213.46s/it]
Next!

## Попробуем такие же деревья, но с Ценой

In [25]:
for i in tqdm([10,25,50,75,100,150,200,300]):
    RF = ens.RandomForestClassifier(n_estimators=i,max_depth=13,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_100_tfidf+price')
    models = add_models(models,RF,res)
    
for i in tqdm(max_depths):
    RF = ens.RandomForestClassifier(n_estimators=75,max_depth=i,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val'],verbose = 2),'svd_100_tfidf+price')
    models = add_models(models,RF,res)
sys.stderr.write('Next!')

100%|██████████| 10/10 [42:24<00:00, 212.14s/it]
Next!

## Пробуем на SVD (10) деревья + Цена

In [26]:
for i in tqdm([10,25,50,75,100,150,200,300]):
    RF = ens.RandomForestClassifier(n_estimators=i,max_depth=13,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, np.hstack((X_svd_10_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_10_tfidf+price')
    models = add_models(models,RF,res)
    
for i in tqdm(max_depths):
    RF = ens.RandomForestClassifier(n_estimators=75,max_depth=i,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, np.hstack((X_svd_10_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val'],verbose = 2),'svd_10_tfidf+price')
    models = add_models(models,RF,res)
sys.stderr.write('Next!')

100%|██████████| 10/10 [11:52<00:00, 64.05s/it]
Next!

### Выцепим из RF-моделей самую лучшую

In [31]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(ens.RandomForestClassifier()))},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True)[:20]:
    print(models[i]['result']['4_level']['score'],'\t',models[i]['model'].get_params()['n_estimators'],'\t',\
          models[i]['model'].get_params()['max_depth'],'\t',models[i]['result']['type_of_fea'])

0.690302962694 	 75 	 50 	 svd_100_tfidf+price
0.688887184306 	 75 	 43 	 svd_100_tfidf+price
0.685931285749 	 75 	 40 	 svd_100_tfidf+price
0.680454450536 	 75 	 50 	 svd_100_tfidf
0.67870170981 	 75 	 43 	 svd_100_tfidf
0.677710784862 	 75 	 35 	 svd_100_tfidf+price
0.676082776803 	 75 	 40 	 svd_100_tfidf
0.666033904251 	 75 	 35 	 svd_100_tfidf
0.657685057257 	 75 	 29 	 svd_100_tfidf+price
0.645591566656 	 75 	 29 	 svd_100_tfidf
0.630846015062 	 75 	 24 	 svd_100_tfidf+price
0.62488348971 	 75 	 23 	 svd_100_tfidf+price
0.618487159763 	 75 	 24 	 svd_100_tfidf
0.612440751541 	 75 	 21 	 svd_100_tfidf+price
0.610733040905 	 75 	 23 	 svd_100_tfidf
0.599039172711 	 75 	 21 	 svd_100_tfidf
0.582255830489 	 75 	 18 	 svd_100_tfidf+price
0.568036966958 	 75 	 18 	 svd_100_tfidf
0.549059348912 	 75 	 15 	 svd_100_tfidf+price
0.532780804344 	 75 	 15 	 svd_100_tfidf
0.532724695979 	 200 	 13 	 svd_100_tfidf+price
0.531855027579 	 300 	 13 	 svd_100_tfidf+price
0.530414297254 	 150 	 13 

### Построим более глубокие деревья (при числе деревьев в 200)

In [ ]:
for i in tqdm([100,75,65,60,55,50]):
    RF = ens.RandomForestClassifier(n_estimators=200,max_depth=i,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(RF, np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_100_tfidf+price')
    models = add_models(models,RF,res)
sys.stderr.write('Next!')

#### Посмотрим на то, какие модели Random Forest обеспечивают наилучший скор на кросс-валидации

In [18]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(ens.RandomForestClassifier()))},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True)[:10]:
    print(models[i]['result']['4_level']['score'],'\t',models[i]['result']['1_level']['score'],'\t',models[i]['model'].get_params()['n_estimators'],'\t',\
          models[i]['model'].get_params()['max_depth'],'\t',models[i]['result']['type_of_fea'])

0.699906260572 	 0.860352032277 	 200 	 55 	 svd_100_tfidf+price
0.699846974883 	 0.860523612408 	 200 	 60 	 svd_100_tfidf+price
0.699802068939 	 0.860157968938 	 200 	 75 	 svd_100_tfidf+price
0.699785682364 	 0.860466380515 	 200 	 65 	 svd_100_tfidf+price
0.699667215839 	 0.86048076054 	 200 	 100 	 svd_100_tfidf+price
0.699501837404 	 0.859880132825 	 200 	 50 	 svd_100_tfidf+price
0.690302962694 	 0.854681123439 	 75 	 50 	 svd_100_tfidf+price
0.688887184306 	 0.853339003735 	 75 	 43 	 svd_100_tfidf+price
0.685931285749 	 0.851475974198 	 75 	 40 	 svd_100_tfidf+price
0.680454450536 	 0.85192738949 	 75 	 50 	 svd_100_tfidf


> Выберем Random Forest с 200 деревьями и с максимальной глубиной 55

## ExtraTrees

In [37]:
n_ests = [50,75,100,125,150,175,200]
n_ests.reverse()
for i in tqdm(n_ests):
    ET = ens.ExtraTreesClassifier(n_estimators=i,max_depth=10,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(ET, np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_100_tfidf+price')
    models = add_models(models,ET,res)
sys.stderr.write('Next!')

100%|██████████| 7/7 [06:05<00:00, 44.73s/it]
Next!

In [38]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(ens.ExtraTreesClassifier()))},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True):
    print(models[i]['result']['4_level']['score'],'\t',models[i]['model'].get_params()['n_estimators'],'\t',\
          models[i]['model'].get_params()['max_depth'],'\t',models[i]['result']['type_of_fea'])

0.553835655191 	 200 	 10 	 svd_100_tfidf+price
0.552021854436 	 175 	 10 	 svd_100_tfidf+price
0.548802110721 	 150 	 10 	 svd_100_tfidf+price
0.546083288366 	 125 	 10 	 svd_100_tfidf+price
0.542403814246 	 100 	 10 	 svd_100_tfidf+price
0.539760566573 	 75 	 10 	 svd_100_tfidf+price
0.528024437864 	 50 	 10 	 svd_100_tfidf+price


In [39]:
md = [15,20,25,30,40,50,75]
md.reverse()
for i in tqdm(md):
    ET = ens.ExtraTreesClassifier(n_estimators=200,max_depth=i,n_jobs=-1)
    res = result_for_avito(models['cross_val'],train.category_id, \
                               cv.cross_val_predict(ET, np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))\
                                                    ,train.category_id,n_jobs=1,\
                                            cv = models['cross_val']),'svd_100_tfidf+price')
    models = add_models(models,ET,res)
sys.stderr.write('Next!')

100%|██████████| 7/7 [16:50<00:00, 130.50s/it]
Next!

In [45]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(ens.ExtraTreesClassifier()))},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True):
    print(models[i]['result']['4_level']['score'],'\t',models[i]['model'].get_params()['n_estimators'],'\t',\
          models[i]['model'].get_params()['max_depth'],'\t',models[i]['result']['type_of_fea'])

0.695371249325 	 200 	 75 	 svd_100_tfidf+price
0.694676709922 	 200 	 50 	 svd_100_tfidf+price
0.688764728199 	 200 	 40 	 svd_100_tfidf+price
0.666610198248 	 200 	 30 	 svd_100_tfidf+price
0.644858121232 	 200 	 25 	 svd_100_tfidf+price
0.620340076667 	 200 	 20 	 svd_100_tfidf+price
0.592267840018 	 200 	 15 	 svd_100_tfidf+price
0.553835655191 	 200 	 10 	 svd_100_tfidf+price
0.552021854436 	 175 	 10 	 svd_100_tfidf+price
0.548802110721 	 150 	 10 	 svd_100_tfidf+price
0.546083288366 	 125 	 10 	 svd_100_tfidf+price
0.542403814246 	 100 	 10 	 svd_100_tfidf+price
0.539760566573 	 75 	 10 	 svd_100_tfidf+price
0.528024437864 	 50 	 10 	 svd_100_tfidf+price


> Выберем ExtraTreesClassifier с 200 деревьями и глубиной 75

### Сливаний файлов с моделями в один

In [14]:
f = open('models2.pkl','wb')
pkl.dump(models,f)
f.close()

f = open('/home/ivanitskiy/nas_storage/projects/houses/RF_25/folder3/models1.pkl','rb')
models1 = pkl.load(f)
f.close()
f = open('models2.pkl','rb')
models2 = pkl.load(f)
f.close()

for i in models2:
    models1.update([(i,models2[i])])

f = open('my_current_models','wb')
pkl.dump(models1,f)
f.close()

f = open('my_current_models','rb')
models = pkl.load(f)
f.close()

## Стакнем результаты лучшей линейной модели и лучших деревьев

In [14]:
import xgboost as xgb

In [15]:
def cv_predict_proba(models,model,train,target):
    res_proba = np.zeros([train.shape[0],target.nunique()])
    for train_fold,test_fold in tqdm(models['cross_val']):
        model.fit(train[train_fold,:],target[train_fold])
        res_proba[test_fold,:] = model.predict_proba(train[test_fold,:])
    return res_proba

In [ ]:
res_proba_LR = cv_predict_proba(models,lin.LogisticRegression(C=3.9,fit_intercept = True,\
                                max_iter = 100,n_jobs = -1),X_tfidf_tr,train.category_id)
res_proba_RF = cv_predict_proba(models,ens.RandomForestClassifier(max_depth = 55, n_estimators=200,n_jobs = -1),\
                                np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))  ,train.category_id)
res_proba_ET = cv_predict_proba(models,ens.ExtraTreesClassifier(n_estimators=200,max_depth=75,n_jobs=-1),\
                                np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))  ,train.category_id)
sys.stderr.write('Next!')

100%|██████████| 5/5 [02:50<00:00, 34.08s/it]
Next!

In [ ]:
X_data = np.hstack((res_proba_ET,np.hstack((res_proba_RF,\
            np.hstack((res_proba_LR,np.array(train.price).reshape([res_proba_LR.shape[0],1])))))))

In [ ]:
pd.DataFrame(X_data).to_pickle('/home/dac/nas_storage/projects/houses/RF_25/X_data')

In [16]:
X_data = pd.read_pickle('/home/dac/nas_storage/projects/houses/RF_25/X_data')

In [17]:
X_data = np.array(X_data)

In [61]:
stack_model = xgb.XGBClassifier(learning_rate=0.1,max_depth=5,n_estimators=100)

res_ = np.zeros([train.shape[0],1])
for train_fold,test_fold in tqdm(models['cross_val']):
    stack_model.fit(\
            X_data[train_fold,:],train.category_id[train_fold])
    res_[test_fold,0] = stack_model.predict(X_data[test_fold,:])
res = result_for_avito(models['cross_val'],train.category_id, res_.reshape(1,res_.shape[0])[0],'stacked')
models = add_models(models,stack_model,res)

100%|██████████| 5/5 [2:21:49<00:00, 1704.79s/it]


In [62]:
stack_model = xgb.XGBClassifier(learning_rate=0.5,max_depth=5,n_estimators=100)

res_ = np.zeros([train.shape[0],1])
for train_fold,test_fold in tqdm(models['cross_val']):
    stack_model.fit(\
            X_data[train_fold,:],train.category_id[train_fold])
    res_[test_fold,0] = stack_model.predict(X_data[test_fold,:])
res = result_for_avito(models['cross_val'],train.category_id, res_.reshape(1,res_.shape[0])[0],'stacked')
models = add_models(models,stack_model,res)

100%|██████████| 5/5 [2:20:47<00:00, 1685.48s/it]


In [37]:
for i in sorted({i:models[i] for i in {k:models[k] for k in models if (k != 'cross_val')} \
               if (type(models[i]['model']) == type(xgb.XGBClassifier()))},\
              key = lambda x: models[x]['result']['4_level']['score'],reverse=True):
    print(models[i]['result']['1_level']['score'],'\t',\
          models[i]['result']['2_level']['score'],'\t',\
          models[i]['result']['3_level']['score'],'\t',\
          models[i]['result']['4_level']['score'],'\t',models[i]['model'].get_params()['n_estimators'],'\t',\
          models[i]['model'].get_params()['learning_rate'],'\t',models[i]['result']['type_of_fea'])

0.954816568176 	 0.927881860242 	 0.854013201358 	 0.84946791478 	 100 	 0.1 	 stacked
0.894498077923 	 0.848622368508 	 0.754475871916 	 0.748843579488 	 100 	 0.5 	 stacked


> Как мы видим, использование стэка на вероятностях лучших моделей помогло на кросс-валидации немного увеличить скор

> Для построения финального прогноза нами будет использоваться такая модель:
- Логистическая регрессия с коэф регуляризации С =  3.9 и обучением свободного члена (на tf-idf матрице)
- Случайный лес с количеством деревьев 200 и с  их глубиной 55 (на разложенной на 100 компонентов tf-idf матрице с добавлением цены лота)
- ExtraTrees классификатор с количеством деревьев 200 и их глубиной 75 (на разложенной на 100 компонентов tf-idf матрице с добавлением цены лота)

> Над предсказанием вероятности по этим трем моделям будет строиться классификатор на основе градиентного бустинга:
- c learning_rate = 0.1, числом деревьев внутри 100 и максимальной глубиной этих деревьев 5

> Данная модель на пятифолдовой кросс-валидации оеспечивает следующее значение accuracy:

>> На самом подробном уровне: 84,95%

>> На следующем уровне:  85,40%

>> На следующем уровне: 92,79%

>> На уровне c самым крупным разбиением: 95,48%

# Построим выбранную модель и сделаем предсказание

In [19]:
LR = lin.LogisticRegression(C=3.9,fit_intercept = True,max_iter = 100,n_jobs = -1,verbose=1)
LR.fit(X_tfidf_tr,train.category_id)
sys.stderr.write('Next!')
f = open('LR','wb')
pkl.dump(LR,f)
f.close()

f = open('LR','rb')
LR = pkl.load(f)
f.close()

RF = ens.RandomForestClassifier(max_depth = 55, n_estimators=200,n_jobs = -1,verbose=2)
RF.fit(np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))  ,train.category_id)
sys.stderr.write('Next!')

ET = ens.ExtraTreesClassifier(n_estimators=200,max_depth=75,n_jobs=-1,verbose=2)
ET.fit(np.hstack((X_svd_tr,np.array(train.price).reshape([train.shape[0],1])))  ,train.category_id)
sys.stderr.write('Next!')

LR_test = LR.predict_proba(X_tfidf_te)
RF_test = RF.predict_proba(np.hstack((X_svd_te,np.array(test.price).reshape([test.shape[0],1]))))
ET_test = ET.predict_proba(np.hstack((X_svd_te,np.array(test.price).reshape([test.shape[0],1]))))
sys.stderr.write('Next!')

X_data_test = np.hstack((ET_test,np.hstack((RF_test,\
            np.hstack((LR_test,np.array(test.price).reshape([LR_test.shape[0],1])))))))
sys.stderr.write('Next!')

stack_model = xgb.XGBClassifier(learning_rate=0.1,max_depth=8,n_estimators=100)
stack_model.fit(X_data,train.category_id)
sys.stderr.write('Next!')

predicted_values = stack_model.predict(X_data_test)
sys.stderr.write('Next!')

result = pd.DataFrame()
result['item_id'] = test.item_id
result['category_id'] = predicted_values

result.to_csv('test_category.csv',index = False)

[LibLinear]

Next!